In [3]:
!pip install pyquaternion

In [51]:
from bokeh.io import output_notebook, show, curdoc
curdoc().theme = 'light_minimal'
output_notebook()
from bokeh.plotting import figure
 
import numpy as np
import numpy.matlib
from numpy import random

from scipy.integrate import solve_ivp
from scipy.optimize import minimize, fminbound
from scipy.interpolate import pchip_interpolate

import copy

from pyquaternion import Quaternion

import math

import warnings

# Attitude Determiniation and Control Systems

## Kinematics Equation

In [5]:
def kinematics(attitude = Quaternion(1), rate=np.array((0.0, 0.0, 0.0))):
  return attitude.derivative(rate)

In [6]:
attitude = Quaternion(1)
rate = np.radians(np.array((0.2, 0.0, 0.0)))
print(rate)

attitude_derivative = kinematics(attitude, rate)

[0.00349066 0.         0.        ]


In [7]:
# Quaternion kinematics are handled by the Quaternion class
print("\nattitude derivative: {0}".format(attitude_derivative))


attitude derivative: +0.000 +0.002i +0.000j +0.000k


## Dynamics Equation

In [8]:
def dynamics(inertia=np.eye(3), torques=np.array((0.0, 0.0, 0.0)), rate=np.array((0.0, 0.0, 0.0)), angular_momenta=np.array((0.0, 0.0, 0.0))):
  return np.linalg.inv(inertia) @ (torques - np.cross(rate, inertia @ rate + angular_momenta))

In [9]:
inertia = 2000e-6 * np.eye(3)

torques = np.array((1e-6, 0.0, 0.0))
angular_momenta = np.array((0.0, 0.1, 0.0))

rate_derivative = dynamics(inertia, torques, rate, angular_momenta)

In [10]:
print("inertia:\n{0}".format(inertia))
print("\nattitude: {0}".format(attitude))
print("\nrate: {0}".format(np.degrees(rate)))

# dynamics are defined by above function
print("\nrate derivative: {0}".format(rate_derivative))

inertia:
[[0.002 0.    0.   ]
 [0.    0.002 0.   ]
 [0.    0.    0.002]]

attitude: +1.000 +0.000i +0.000j +0.000k

rate: [0.2 0.  0. ]

rate derivative: [ 0.0005      0.         -0.17453293]


## pFDA Model

In [36]:
class Actuator:
  """Actuator base class.
  """
  def __init__(self, alignment=np.array((1.0, 0.0, 0.0)), angular_momentum=0, input=0, torque=0):
    self.alignment = alignment
    self.angular_momentum = angular_momentum
    self.input = input
    self.torque = torque

  @property
  def alignment(self):
    return self._alignment

  @alignment.setter
  def alignment(self, new):
    self._alignment = new

  @property
  def angular_momentum(self):
    return self._angular_momentum * self._alignment

  @angular_momentum.setter
  def angular_momentum(self, new):
    self._angular_momentum = new

  @property
  def input(self):
    return self._input

  @input.setter
  def input(self, new):
    self._input = new

  @property
  def torque(self):
    return self._torque * self._alignment

  @torque.setter
  def torque(self, new):
    self._torque = new

In [63]:
actuator = Actuator(torque=1e-6)
print(actuator.torque)
print(actuator.angular_momentum)
actuator.input(1)

[1.e-06 0.e+00 0.e+00]
[0. 0. 0.]


TypeError: ignored

In [58]:
class Pfda(Actuator):
  def __init__(self, alignment=np.array((1.0, 0.0, 0.0)), angular_momentum=0, gain=1, input=0, time_constant=1, torque=0):
    super().__init__(alignment=alignment, angular_momentum=angular_momentum, input=input, torque=torque)
    self.gain = gain
    self.time_constant = time_constant

  @property
  def gain(self):
    return self._gain

  @gain.setter
  def gain(self, new):
    if new > 0:
      self._gain = new
    else:
      raise ValueError('Negative gain is not allowed!')

  @property
  def input(self):
    return self._input
    
  @input.setter
  def input(self, new):
    if new < -1:
      self._input = -1
      warnings.warn("Input less than -1")
    elif new > 1:
      self._input = 1
      warnings.warn("Input greater than 1")
    else:
      self._input = new
  
  @property
  def time_constant(self):
    return self._time_constant

  @time_constant.setter
  def time_constant(self, new):
    if new > 0:
      self._time_constant = new
    else:
      raise ValueError('Negative time constant is not allowed')

In [62]:
pfda = Pfda(gain=80e-6, time_constant=0.5)
print(pfda.gain)
print(pfda.time_constant)
print(pfda.input)

8e-05
0.5
0


In [45]:
def pfda(t, x, u, p):
  '''pFDA dynamics'''
  return np.array([x[1] + p[0]/p[1] * u, -1/p[1] * (x[1] + p[0]/p[1] * u)]).flatten()

pa = [80e-6, 0.5]   # pFDA parameters
ua = 1              # control

# solve the initial value problem
sol = solve_ivp(pfda, [0, 5], [0, 0], args=(ua, pa), first_step=0.1, max_step=0.1)

# grab simulation output and create output vector from simulated state vector
t = sol.t
xa = sol.y
ya = copy.deepcopy(xa)
ya[1,:] = ya[1,:] + pa[0]/pa[1]*ua

## Spacecraft Object

In [46]:
class AngularRate:
  def __init__(self, rate=(0.0, 0.0, 0.0)):
    self._rate = np.array(rate)

  @classmethod
  def from_degrees(cls, rate):
    """Initialize from angular rate vector in degree per second.
    """
    return cls(np.radians(np.array(rate)))

  def __str__(self):
      """An informal, nicely printable string representation of the AngularRate object.
      """
      return "{:.3f} {:.3f} {:.3f}".format(self._rate[0], self._rate[1], self._rate[2])

  def __repr__(self):
    """The 'official' string representation of the AngularRate object.

    This is a string representation of a valid Python expression that could be
    used to recreate an object with the same value (given an appropriate
    environment)
    """
    return "AngularRate({!r}, {!r}, {!r})".format(self._rate[0], self._rate[1], self._rate[2])

  def derivative(self, inertia=np.eye(3), torques=np.array((0.0, 0.0, 0.0)),angular_momenta=np.array((0.0, 0.0, 0.0))):
    return np.linalg.inv(inertia) @ (torques - np.cross(self._rate, inertia @ self._rate + angular_momenta))

  def update(self, inertia=np.eye(3), torques=np.array((0.0, 0.0, 0.0)), angular_momenta=np.array((0.0, 0.0, 0.0))):
    # TODO: implement update, maybe based on ivp_solve?
    # NOTE: How to deal with parallel updates for actuators and stuff?
    raise NotImplementedError()

  @property
  def rate(self):
    return self._rate

In [47]:
class Spacecraft:
  def __init__(self, inertia=np.eye(3), attitude=Quaternion(1), rate=AngularRate(), disturbances=[None], actuators=[None], controller=None,):
    self._inertia = inertia
    self._attitude = attitude
    self._rate = rate
    self._actuators = actuators

  def update(self):
    """TODO: implement update of attitude, angular rate, and all actuators.
    """
    raise NotImplementedError

  @property
  def attitude(self):
    return self._attitude

  @property
  def rate(self):
    return self._rate

In [48]:
rate_degrees = (0.0, 1, 0.0)

spacecraft = Spacecraft(rate=AngularRate.from_degrees(rate_degrees))
print(spacecraft.attitude)
print(spacecraft.rate)
spacecraft.update()

1.000 +0.000i +0.000j +0.000k
0.000 0.017 0.000


NotImplementedError: ignored